In [2]:
import pandas as pd
import numpy as np 
import plotly.express as px
from tqdm import tqdm 
tqdm.pandas() 
import torch

C:\Users\romeo\anaconda3\lib\site-packages\tqdm\std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [1]:
a = 1
b = 2
print(a+b)

3


In [3]:
from sentence_transformers import SentenceTransformer, util
import numpy as np

ModuleNotFoundError: No module named 'sentence_transformers'

In [2]:


model_path = '../../Checkpoint/KoSBERT/kosbert-klue-bert-base'

embedder = SentenceTransformer(model_path) 

In [3]:
data = pd.read_csv('../data/key_word.csv')
data = data.iloc[:6005,:]
data2 = pd.read_csv('../data/key_word_sbc.csv')

data = data.append(data2)
data = data.reset_index(drop=True)
print(data.shape) 
del data2

data.key_word = data.key_word.apply(lambda x: str(x))
data.head()

(9372, 3)


/var/folders/8g/dtz5rgsx24l71h158wfj1f_r0000gn/T/ipykernel_93673/4704144.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(data2)


,imsi_no,key_word,label
0,788D3D6BD0EC8316E9DB49535D449D6F51BCE703,탬,0
1,908AFC20E2CA5B6309E2203A4DF52353485BFAE7,참치액,1
2,135AEC6F05FECB8BBFBEB25ABA846AE343E91F27,도장,1
3,B59ED095E4D6F59FCAC2584478A05FB0B8C60B19,스캐너,1
4,B25A2422B339E610FE137DF884CBA0EE1F897486,로베르타 가방,1


In [4]:
emd_x = embedder.encode(list(data.key_word), convert_to_tensor=True)
emd_x.shape

torch.Size([9372, 768])

In [5]:
x_train = emd_x[:7500]
x_test = emd_x[7500:]
y_train = data.label.iloc[:7500]
y_train = torch.FloatTensor(y_train)
y_test = data.label.iloc[7500:].values
y_test = torch.FloatTensor(y_test) 

In [6]:
import torch
from torch import nn
import torch.nn.functional as F

In [7]:
class BertClassifier(nn.Module):
    def __init__(self):
        super(BertClassifier, self).__init__()
        self.classfier1 = nn.Linear(768, 128)
        self.classfier2 = nn.Linear(128, 1)
        self.dropout = nn.Dropout(0.2)
        
    def forward(self, emd_x):
        emd_x = self.dropout(emd_x)
        emd_x = self.classfier1(emd_x)  
        emd_x = self.classfier2(emd_x)
        emd_x = torch.sigmoid(emd_x)
        
        return emd_x

In [8]:
model = BertClassifier()
criterion = nn.BCELoss() 
optimizer =  torch.optim.Adam(model.parameters(), lr= 2e-5) 
epochs = 2000

In [9]:
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    pred_y = model(x_train)
    loss = criterion(pred_y.squeeze(), y_train)
    
    if epoch % 100 == 0:
        print('Train loss at {} is {}'.format(epoch, loss.item()))
    loss.backward()
    optimizer.step() 

Train loss at 0 is 0.7092633843421936
Train loss at 100 is 0.5292726159095764
Train loss at 200 is 0.4545319974422455
Train loss at 300 is 0.4188496172428131
Train loss at 400 is 0.3971322774887085
Train loss at 500 is 0.3878142833709717
Train loss at 600 is 0.38128775358200073
Train loss at 700 is 0.3733174502849579
Train loss at 800 is 0.3683329224586487
Train loss at 900 is 0.365113228559494
Train loss at 1000 is 0.3628239035606384
Train loss at 1100 is 0.36117103695869446
Train loss at 1200 is 0.36217883229255676
Train loss at 1300 is 0.35712987184524536
Train loss at 1400 is 0.36000722646713257
Train loss at 1500 is 0.3579857051372528
Train loss at 1600 is 0.35379940271377563
Train loss at 1700 is 0.35627880692481995
Train loss at 1800 is 0.35437512397766113
Train loss at 1900 is 0.35114458203315735


In [10]:
pred_y = model(x_test)
loss = criterion(pred_y.squeeze(), y_test)
print('Test acc: {}'.format(sum(torch.round(pred_y.squeeze())==y_test).item()))
print('Test loss: {}'.format(loss)) 

Test acc: 1559
Test loss: 0.38627973198890686


In [11]:
pred_y =  model(emd_x).squeeze()
data['prob'] = pred_y.detach().numpy()
data['pred'] = torch.round(pred_y).detach().numpy() 
data.head(30) 

,imsi_no,key_word,label,prob,pred
0,788D3D6BD0EC8316E9DB49535D449D6F51BCE703,탬,0,0.202483,0.0
1,908AFC20E2CA5B6309E2203A4DF52353485BFAE7,참치액,1,0.980068,1.0
2,135AEC6F05FECB8BBFBEB25ABA846AE343E91F27,도장,1,0.877223,1.0
3,B59ED095E4D6F59FCAC2584478A05FB0B8C60B19,스캐너,1,0.545194,1.0
4,B25A2422B339E610FE137DF884CBA0EE1F897486,로베르타 가방,1,0.979526,1.0
5,9C5D6F78EC0D4971953D5DECB2E4C7638000DBD6,야간 주급,0,0.215196,0.0
6,B66E0D57CBDA555779053FC3E4F8EF47C2000322,명 쇱,0,0.038493,0.0
7,782BA5086AA4556D01E45F248905F670646C03A2,우산,1,0.858635,1.0
8,06F844A2100125C4C7220B752A33CB976F1EF445,ed8ca8,0,0.258294,0.0
9,C23A87DC0FA9394ECBEF41E3AB8B04B2D3DD1C4F,강아지간식,1,0.990669,1.0


In [12]:
from sklearn.metrics import confusion_matrix, classification_report

cm = confusion_matrix(data.label, data.pred)
report = classification_report(data.label, data.pred)
print(cm)
print(report) 

[[3244  665]
 [ 788 4675]]
              precision    recall  f1-score   support

           0       0.80      0.83      0.82      3909
           1       0.88      0.86      0.87      5463

    accuracy                           0.84      9372
   macro avg       0.84      0.84      0.84      9372
weighted avg       0.85      0.84      0.85      9372

